In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from tooluniverse.execute_function import ToolUniverse
import pdb
# from langchain.agents import AgentExecutor, Tool

# PrimeKG

In [ ]:
# primekg = pd.read_csv('kg.csv', low_memory=True)
# primekg.query('y_type=="contraindication"|x_type=="fosaprepitant"')

In [ ]:
# primekg.query('y_type=="disease"|x_name=="fosaprepitant"')

# ToolUniverse

In [2]:

engine = ToolUniverse()
engine.load_tools()
tool_name_list, tool_desc_list = engine.refresh_tool_name_desc()

# print("Available tools:")
# for name, desc in zip(tool_name_list, tool_desc_list):
#     print(f"- {name}: {desc}")


Tool files:
{'opentarget': '/Users/awxlong/anaconda3/envs/ai/lib/python3.8/site-packages/tooluniverse/data/opentarget_tools.json', 'fda_drug_label': '/Users/awxlong/anaconda3/envs/ai/lib/python3.8/site-packages/tooluniverse/data/fda_drug_labeling_tools.json', 'special_tools': '/Users/awxlong/anaconda3/envs/ai/lib/python3.8/site-packages/tooluniverse/data/special_tools.json', 'monarch': '/Users/awxlong/anaconda3/envs/ai/lib/python3.8/site-packages/tooluniverse/data/monarch_tools.json'}
Number of tools before load tools: 0
Number of tools after load tools: 214


In [3]:
result = engine.run_one_function({
    "name": "get_contraindications_by_drug_name",
    "arguments": {
        "drug_name": "fosaprepitant",
        "limit": 5,
        "skip": 0
    }
})

print(result)

loaded function call json {'name': 'get_contraindications_by_drug_name', 'arguments': {'drug_name': 'fosaprepitant', 'limit': 5, 'skip': 0}}
Initiating callable_function from loaded tool dicts.
https://api.fda.gov/drug/label.json?limit=5&skip=0&search=(openfda.brand_name:(fosaprepitant)+openfda.generic_name:(FOSAPREPITANT))+AND+(_exists_:contraindications)+AND+(_exists_:openfda.brand_name+_exists_:openfda.generic_name)
{'meta': {'skip': 0, 'limit': 5, 'total': 27}, 'results': [{'openfda.brand_name': ['FOSAPREPITANT'], 'openfda.generic_name': ['FOSAPREPITANT'], 'contraindications': ['4 CONTRAINDICATIONS Fosaprepitant for injection is contraindicated in patients: who are hypersensitive to any component of the product. Hypersensitivity reactions including anaphylactic reactions, flushing, erythema, and dyspnea have been reported [see Warnings and Precautions (5.2) , Adverse Reactions (6.2) ] . taking pimozide. Inhibition of CYP3A4 by aprepitant, the active moiety, could result in elevated

In [ ]:
# model.get_input_embeddings()

In [4]:
import re
def extract_drugs_from_question(question, options):
    """
    Extracts a potential drug name from the 'question' or 'options' fields of a quiz-like input dictionary.
    Returns the drug name as a string, or None if not found.
    """
    # Search in question
    # question = input_dict.get("question", "")
    
    # Drug names are often lowercase (ending in -ine/-cin/-one/-tan/-ide/-vir, etc.), 
    # but allowing for case and general pattern: at least 5-letter, not common English words.
    drug_pattern = re.compile(r'\b([A-Za-z]{5,})\b', re.IGNORECASE)
    
    # For this example, look for words followed by 'for injection' or 'tablet', etc.
    # So, find "xxxx for injection"
    match = re.search(r'([A-Za-z0-9\-]{5,})\s+(?:for|as|in)\s+[^\s]+', question, re.IGNORECASE)
    if match:
        return [match.group(1).lower()]  # you can return as-is if you prefer the original case
    
    # Otherwise, fall back to just scanning for long words in the question
    words = drug_pattern.findall(question)
    # Very basic filter: exclude common English words
    # List of possible filter words can be expanded as needed
    exclusions = {"patients", "during", "after", "receiving", "immediately", "their", "healthcare", "provider", "should", "severe", "allergic", "reactions", "and", "seek", "emergency", "medical", "care", "symptoms", "resolve", "wait", "stop", "treatment", "permanently", "take", "over", "counter", "antihistamines", "chemotherapy", "with", "the", "they", "experience", "on", "own", "or", "if", "do", "what"}
    for word in words:
        word_lower = word.lower()
        if word_lower not in exclusions and not word_lower.endswith("ing"):
            return list(word)   # Return the first word that looks like a drug name

    # Optionally: search in options
    for opt in options:
        words = drug_pattern.findall(opt)
        for word in words:
            if word.lower() not in exclusions:
                return word

    # Nothing found
    return None



In [5]:
import json

with open('data/curebench_valset_pharse1_mini.jsonl', 'r', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]


In [6]:
drugs = extract_drugs_from_question(data[1]['question'], data[1]['options'])
type(drugs)
drugs

['fosaprepitant']

In [7]:
# Configuration
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
DEVICE = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"


In [8]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
# 3. Molecular feature extractor using RDKit
FINGER_PRINT_SIZE = 512  # Size of the fingerprint vector
class MolecularFeatureExtractor:
    def __init__(self, fingerprint_size=FINGER_PRINT_SIZE):
        self.fingerprint_size = fingerprint_size
        
    def smiles_to_fingerprint(self, smiles):
        """Convert SMILES to Morgan fingerprint"""
        mol = Chem.MolFromSmiles(smiles)
        # pdb.set_trace()
        if mol:
            return AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=self.fingerprint_size)
        return np.zeros(self.fingerprint_size)
    
    def drug_name_to_features(self, drug_name):
        """Extract molecular features from drug name"""
        # Query PrimeKG for SMILES notation
        if drug_name == 'fosaprepitant':
            return self.smiles_to_fingerprint('C[C@H](C1=CC(=CC(=C1)C(F)(F)F)C(F)(F)F)O[C@@H]2[C@@H](N(CCO2)CC3=NC(=NN=C3P(=O)(O)O)O)C4=CC=C(C=C4)F')
        else:
            return np.zeros(self.fingerprint_size)
    
    def drug_name_to_smile(self, drug_name):
        if drug_name == 'fosaprepitant':
            return 'C[C@H](C1=CC(=CC(=C1)C(F)(F)F)C(F)(F)F)O[C@@H]2[C@@H](N(CCO2)CC3=NC(=NN=C3P(=O)(O)O)O)C4=CC=C(C=C4)F'
        else:
            return 'C[C@H]'


In [9]:
class MolecularGraphBuilder:
    def __init__(self):
        self.similarity_threshold = 0.6
        
    def calculate_similarity(self, fp1, fp2):
        """Calculate Tanimoto similarity between fingerprints"""
        if np.all(fp1 == 0) or np.all(fp2 == 0):
            return 0.0
        intersection = np.sum(fp1 & fp2)
        union = np.sum(fp1 | fp2)
        return intersection / union if union > 0 else 0.0
    
    def build_molecular_graph(self, drug_features):
        """Construct molecular graph with edges based on structural similarity"""
        num_drugs = len(drug_features)
        
        # Create empty graph for no drugs
        if num_drugs == 0:
            return {
                "features": torch.zeros(1, FINGER_PRINT_SIZE, dtype=torch.float32),
                "edge_index": torch.empty((2, 0), dtype=torch.long)
            }
        
        # Convert features to tensor
        features = torch.tensor(np.array(drug_features), dtype=torch.float32)
        
        # Calculate similarity matrix
        similarity_matrix = np.zeros((num_drugs, num_drugs))
        for i in range(num_drugs):
            for j in range(i+1, num_drugs):
                similarity = self.calculate_similarity(
                    drug_features[i], drug_features[j]
                )
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity
        
        # Create edges based on similarity threshold
        edge_list = []
        for i in range(num_drugs):
            for j in range(i+1, num_drugs):
                if similarity_matrix[i, j] > self.similarity_threshold:
                    edge_list.append([i, j])
        
        # Convert to edge index
        if edge_list:
            edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
        else:
            # Create self-loops if no edges
            edge_index = torch.tensor([[i for i in range(num_drugs)], 
                                     [i for i in range(num_drugs)]], 
                                    dtype=torch.long)
        
        return {
            "features": features,
            "edge_index": edge_index
        }

In [10]:
import gc

gc.collect()     # force Python garbage collection
torch.mps.empty_cache()  # clear MPS cached memory

In [11]:

# 1. Initialize model with QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

llm = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    # quantization_config=bnb_config, # deconfigurate in the cloud
    device_map="auto",
    # trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

In [39]:
tool_desc_list[:3]

['get_associated_targets_by_disease_efoId: Find targets associated with a specific disease or phenotype based on efoId.',
 'get_associated_diseases_phenotypes_by_target_ensemblID: Find diseases or phenotypes associated with a specific target using ensemblId.',
 'target_disease_evidence: Explore evidence that supports a specific target-disease association. Input is disease efoId and target ensemblID.']

# Manual labeling my dataset (adding reasoning traces)

In [50]:
len(str(tool_desc_list))
# tool_name_list

27220

In [49]:
IDX = 0
prompt = f"""<|system|> You are a clinical reasoning agent. Please answer the question. </s> <|user|> Question: {data[IDX]['question']} Options: {data[IDX]['options']} </s><|assistant|>\n"""
# Generate tool-using response
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
outputs = llm.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.95,
    num_beams=1, # TODO: Change in the cloud
    pad_token_id=tokenizer.eos_token_id,
    num_return_sequences=1
    # max_new_tokens=256,
    # temperature=0.7,
    # top_p=0.9,
    # num_beams=3,
    # early_stopping=True
)

In [46]:
tokenizer.decode(outputs[0], skip_special_tokens=True) # CoT

"<|system|> You are a clinical reasoning agent. Please answer the question. </s> <|user|> Question: Which group had the highest percentage of patients reporting headache in the Phase 3 clinical study? Options: {'A': 'GOPRELTO 4% solution group', 'B': 'Cocaine Hydrochloride 8% solution group', 'C': 'Placebo group', 'D': 'None of the groups'} </s><|assistant|>\n <think>\nAlright, so I'm trying to figure out which group in this Phase 3 clinical study had the highest percentage of patients reporting headaches. The options are GOPRELTO 4% solution group, Cocaine Hydrochloride 8% solution group, Placebo group, or none of the groups.\n\nFirst, I need to understand what each of these groups represents. The Placebo group is usually the control group where participants don't receive any active treatment. They serve as a baseline to compare other groups against.\n\nThen there's GOPRELTO, which I think stands for Generalized Poly(dI-Polymers) Relapse Overdose. It's a medication used for relapse pr

In [50]:
tokenizer.decode(outputs[0], skip_special_tokens=True) # 

'<|system|> You are a clinical reasoning agent. Please answer the question. </s> <|user|> Question: Which group had the highest percentage of patients reporting headache in the Phase 3 clinical study? Options: {\'A\': \'GOPRELTO 4% solution group\', \'B\': \'Cocaine Hydrochloride 8% solution group\', \'C\': \'Placebo group\', \'D\': \'None of the groups\'} </s><|assistant|>\n</think>\n\nTo determine which group had the highest percentage of patients reporting headaches in the Phase 3 clinical study, we analyzed the provided options:\n\n- **A**: "GOPRELTO 4% solution group"\n- **B**: "Cocaine Hydrochloride 8% solution group"\n- **C**: "Placebo group"\n- **D**: "None of the groups"\n\nUpon evaluating the data, the "Placebo group" had the highest percentage of headaches at **4%**. This conclusion aligns with the standard results for Phase 3 clinical studies, where the placebo group serves as a control to assess the efficacy of the active treatment.\n\n**Answer:** The group with the highes

In [14]:
# Decode response
full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract reasoning content
reasoning_match = re.search(r'<think>(.*?)</think>', full_response, re.DOTALL)

In [15]:
reasoning_match

In [41]:
# 7. Enhanced Therapeutic Dataset with Molecular Graphs
import torch
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
# Molecular feature extractor
mol_feature_extractor = MolecularFeatureExtractor()
mol_graph_builder = MolecularGraphBuilder()

class TherapeuticDataset(torch.utils.data.Dataset):
    def __init__(self, samples, max_length=128):
        # TODO: Change max_length when submitting to Kaggle
        self.samples = samples
        self.max_length = max_length
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        
        # Extract drugs
        drugs = extract_drugs_from_question(sample['question'], sample['options'])
        
        # Get molecular features
        mol_features = []
        for drug in drugs: # TODO: SUpport for multiple drugs
            features = mol_feature_extractor.drug_name_to_features(drug)
            mol_features.append(features)
        
        # pdb.set_trace()
        # Build molecular graph with edges
        graph = mol_graph_builder.build_molecular_graph(mol_features)
        
        # Format input with tools
        
        prompt = f"""<|system|>\nYou are a clinical reasoning agent. Please answer the question: <|user|>\nQuestion: {sample['question']} Options: {json.dumps(sample['options'])}\n<|assistant|>\n{sample['correct_answer']}: {sample['options'][sample['correct_answer']]}\n"""

        # Tokenizer has default left padding
        inputs = tokenizer(
            prompt,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        # Create labels: mask input portion, keep answer portion
        input_text = prompt.split("<|assistant|>")[0] + "<|assistant|>\n"
        input_tokens = tokenizer(input_text, return_tensors="pt")["input_ids"]
        labels = inputs["input_ids"].clone()
        
        # Mask everything before the assistant's answer
        mask_position = min(input_tokens.shape[1], self.max_length)
        labels[:, :mask_position] = -100
        # pdb.set_trace()
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels.squeeze(),
            "molecular_features": graph["features"].to(DEVICE),
            "molecular_edges": graph["edge_index"].to(DEVICE),
            "question": sample["question"],
            "options": sample["options"],
            "drugs": drugs,
            "correct_answer": sample["correct_answer"],
            "prompt": prompt
        }


In [24]:
# 8. Multimodal Fusion Model (GNN + LLM)
class MultimodalTherapeuticModel(nn.Module):
    def __init__(self, llm, gnn_output_size=256, molecular_dim=FINGER_PRINT_SIZE):
        super().__init__()
        self.llm = llm
        self.molecular_dim = molecular_dim
        
        # GNN for molecular graph processing
        self.gnn = nn.ModuleList([
            GCNConv(molecular_dim, 512),
            nn.ReLU(),
            GCNConv(512, gnn_output_size),
            nn.ReLU()
        ]).to(DEVICE)
        
        # Projection layers
        self.molecular_projection = nn.Linear(gnn_output_size, llm.config.hidden_size).to(DEVICE)
        # self.attention_fusion = nn.MultiheadAttention(
        #     embed_dim=llm.config.hidden_size,
        #     num_heads=4,
        #     batch_first=True
        # ).to(DEVICE)
        
    def forward(self, input_ids, attention_mask, molecular_features, molecular_edges):
        gc.collect()     # force Python garbage collection
        torch.mps.empty_cache()  # clear MPS cached memory
        # Get LLM embeddings
        text_embeds = self.llm.model.embed_tokens(input_ids)
        
        # Process molecular features with GNN
        if molecular_features.dim() == 2:  # Handle single graph
            molecular_features = molecular_features.unsqueeze(0)
        
        mol_embeds_list = []
        for i in range(molecular_features.size(0)):
            mol_data = Data(
                x=molecular_features[i],
                edge_index=molecular_edges[i]
            )
            x = mol_data.x
            for layer in self.gnn:
                if isinstance(layer, GCNConv):
                    x = layer(x, mol_data.edge_index)
                else:
                    x = layer(x)
            mol_embeds_list.append(x)
        
        # Stack and aggregate molecular embeddings
        mol_embeds = torch.stack(mol_embeds_list)
        mol_embeds = torch.mean(mol_embeds, dim=1)  # Average over nodes
        mol_embeds = self.molecular_projection(mol_embeds) # mol_embeds = [batch_size, gnn_output_size -> llm.config.hidden_size] # upsample
        
        # # Fuse modalities with attention # instead do simple concatenation
        # fused_embeds, _ = self.attention_fusion(
        #     query=text_embeds,
        #     key=mol_embeds.unsqueeze(1),
        #     value=mol_embeds.unsqueeze(1)
        # )

        # Concatenate text and molecular embeddings
        fused_embeds = torch.cat((text_embeds, mol_embeds.unsqueeze(1)), dim=1) if len( mol_embeds.size()) == 2 else torch.cat((text_embeds, mol_embeds), dim=1)
        attention_mask = torch.cat((attention_mask, torch.ones(1 , mol_embeds.size(0)).to(DEVICE)), dim=1) # pad attention mask to attend to molecular embedding(s)
        # pdb.set_trace()
        # fused_embeds = text_embeds # debug and see what if happens if we don't do multimodal fusion
        # Pass through LLM via autocast for MPS
        # with torch.autocast(device_type='cuda', dtype=torch.float16): # enable this support on Kaggle
        outputs = self.llm(
            inputs_embeds=fused_embeds,
            attention_mask=attention_mask
        )
        return outputs

In [25]:
# 9. Enhanced Therapeutic Agent with Molecular Graph Processing
class MultimodalTherapeuticAgent:
    def __init__(self, model, tokenizer, tool_engine):
        self.model = model
        self.tokenizer = tokenizer
        self.tool_engine = tool_engine
        self.mol_feature_extractor = MolecularFeatureExtractor()
        self.mol_graph_builder = MolecularGraphBuilder()
        
    def run(self, question, options):
        # Extract drugs and get molecular features
        drugs = extract_drugs_from_question(question, options)
        mol_features = []
        for drug in drugs:
            features = self.mol_feature_extractor.drug_name_to_features(drug)
            mol_features.append(features)
        
        # Build molecular graph
        graph = self.mol_graph_builder.build_molecular_graph(mol_features)
        
        # Create prompt
        prompt = f"""<|System|>\nYou are a clinical reasoning agent. Use available tools to answer the question. Available tools: {tool_desc_list[:3]} <|User|>\nQuestion: {question} Options: {json.dumps(options)}\n<|Assistant|><think>"""
        # Generate initial reasoning
        inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(DEVICE)
        with torch.no_grad():
            outputs = self.model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                molecular_features=graph["features"].to(DEVICE),
                molecular_edges=graph["edge_index"].to(DEVICE)
            )
        reasoning = tokenizer.decode(outputs.logits.argmax(-1)[0], skip_special_tokens=True)
        
        # Parse tool calls
        tool_calls = self._parse_tool_calls(reasoning)
        tool_results = []
        
        # Execute tools
        for call in tool_calls:
            try:
                result = self.tool_engine.run_one_function(call)
                tool_results.append(f"Tool {call['name']} result: {str(result)}")
            except Exception as e:
                tool_results.append(f"Tool {call['name']} error: {str(e)}")
        
        # Generate final answer
        final_prompt = f"{prompt}{reasoning}\nTool Results:\n" + "\n".join(tool_results) + "\nFinal Answer:"
        inputs = tokenizer(final_prompt, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.7,
                top_p=0.9
            )
        final_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        return final_response
    
    def _parse_tool_calls(self, text):
        """Parse tool calls from model output using structured detection"""
        tool_calls = []
        # Pattern: [TOOL: tool_name(arg1=value1, arg2=value2)]
        pattern = r'\[TOOL:\s*(\w+)\s*\((.*?)\)\]'
        matches = re.findall(pattern, text)
        
        for match in matches:
            tool_name = match[0]
            arg_str = match[1]
            args = {}
            
            # Parse arguments
            arg_pairs = [pair.split('=') for pair in arg_str.split(',') if '=' in pair]
            for key, value in arg_pairs:
                args[key.strip()] = value.strip().strip('"').strip("'")
            
            tool_calls.append({"name": tool_name, "arguments": args})
        
        return tool_calls

In [19]:
torch.mps.empty_cache()

In [38]:
torch.mps.empty_cache()
model = MultimodalTherapeuticModel(llm)
# model.to(DEVICE)
torch.mps.empty_cache()

In [ ]:
agent = MultimodalTherapeuticAgent(model=model, tokenizer=tokenizer, tool_engine=engine)

In [42]:
dataset = TherapeuticDataset(samples = data)

In [43]:
dataset[1]


[18:25:09] DEPRECATION WARNING: please use MorganGenerator


{'input_ids': tensor([151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151646,     27,     91,   8948,     91,    397,   2610,
            525,    264,  14490,  32711,   8315,     13,   5209,   4226,    279,
           3405,     25,  82639,    872,     91,    397,  14582,     25,   3555,
           1265,   6835,    653,    421,    807,   3139,  15386,  56496,  24473,
           2337,    476,   1283,  12308,  48390,    391,   9995,  50944,    369,
          25071,     30,  14566,     25,   5212,     32,    788,    330,  14190,
            369,    279,  13495,    311,   8830,    389,    862,   1828,  10465,
            330,     33,    788,    330,  37891,    862,  18478,   9109,   7069,
            323,   5931,  12851,   6457,   2453,  10465,    330,     34,    788,
            330,  10674,  61630,   6380,  30759,  10465,    330,     35,    788,
            330,  17814,    916,  10603,  63189,   3196,   6996,    380,  95981,
           1189

In [28]:
out = model(
    input_ids=dataset[1]['input_ids'].unsqueeze(0).to(DEVICE),
    attention_mask=dataset[1]['attention_mask'].unsqueeze(0).to(DEVICE),
    molecular_features=dataset[1]['molecular_features'].unsqueeze(0).to(DEVICE),
    molecular_edges=dataset[1]['molecular_edges'].unsqueeze(0).to(DEVICE)
)   

[17:02:17] DEPRECATION WARNING: please use MorganGenerator
[17:02:17] DEPRECATION WARNING: please use MorganGenerator
[17:02:17] DEPRECATION WARNING: please use MorganGenerator
[17:02:17] DEPRECATION WARNING: please use MorganGenerator


In [29]:
out['logits'].shape

torch.Size([1, 129, 151936])

In [30]:
tokenizer.decode(out.logits.argmax(-1)[0], skip_special_tokens=True)

")\n\na<><< are given programmer assistant system, You the information and reason the questions. You tools include \nSystem_rankation_resources'],_systemisease','],'],', E all by with a given E using E using on EfoId strings 'get_ated_targetsiseases_byenotype_by_d:efbl:: Find the or phenotypes associated with a specific target in esemblID.', 'get_isease_fo_by Find evidence of a or target target inisease relationship.',']\n a a andfoId and evidence ensemblID. \n\n< Question>\n[: What is I with to they have a hyp reactions-"

# FIX prompting

In [ ]:
out2=out
tokenizer.decode(out2.logits.argmax(-1)[0], skip_special_tokens=True)

# with multimodal fusion

In [ ]:
tokenizer.decode(out.logits.argmax(-1)[0], skip_special_tokens=False)

# only text

In [ ]:
out1=out
tokenizer.decode(out1.logits.argmax(-1)[0], skip_special_tokens=False)

In [ ]:
question = dataset[1]['question']
options = dataset[1]['options']

prompt = f"""<|System|>\nYou are a clinical reasoning agent. Use available tools to answer the question. Available tools: {tool_desc_list} <|User|>\nQuestion: {question} Options: {json.dumps(options)}\n<|Assistant|><think>"""
        # Generate initial reasoning
inputs = tokenizer(prompt, return_tensors="pt", padding=True)

In [ ]:
# Prepare chat messages following DeepSeek's format
messages = [
    {"role": "system", "content": "You are a clinical reasoning agent. Use available tools to answer the question."},
    {"role": "user", "content": f"Question: {question}\nOptions: {json.dumps(options)}"}
]
        
# Apply chat template
input_ids = tokenizer.apply_chat_template(
    messages, 
    add_generation_prompt=True, 
    return_tensors="pt"
)

In [ ]:
input_ids
tokenizer.decode(input_ids[0], skip_special_tokens=True)

# training setup

In [ ]:
from peft import LoraConfig
from trl import SFTTrainer, DPOTrainer

# 9. Distributed Training Setup (for future scaling)
def setup_distributed_training():
    """Configuration for multi-node training"""
    return {
        "strategy": "deepspeed",
        "config": {
            "train_batch_size": "auto",
            "train_micro_batch_size_per_gpu": "auto",
            "gradient_accumulation_steps": "auto",
            "zero_optimization": {
                "stage": 3,
                "offload_optimizer": {
                    "device": "cpu",
                    "pin_memory": True
                },
                "overlap_comm": True,
                "contiguous_gradients": True
            },
            "fp16": {
                "enabled": True,
                "loss_scale": 0,
                "loss_scale_window": 1000,
                "initial_scale_power": 16,
                "hysteresis": 2,
                "min_loss_scale": 1
            },
            "activation_checkpointing": {
                "partition_activations": True,
                "contiguous_memory_optimization": True
            }
        }
    }

# 10. Training Workflow with SFT and DPO
def train_therapeutic_agent(train_samples, eval_samples):
    # Prepare datasets
    train_dataset = TherapeuticDataset(train_samples)
    eval_dataset = TherapeuticDataset(eval_samples)
    
    # LoRA configuration
    peft_config = LoraConfig(
        r=32,
        lora_alpha=64,
        target_modules=["q_proj", "k_proj", "v_proj"],
        lora_dropout=0.05,
        task_type="CAUSAL_LM"
    )
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir="./therapeutic_agent",
        per_device_train_batch_size=2,  # Reduced for multimodal model
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=1e-5,
        num_train_epochs=3,
        fp16=True,
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        report_to="none",
        deepspeed=setup_distributed_training()["config"] if torch.cuda.device_count() > 1 else None
    )
    
    # Supervised Fine-Tuning
    sft_trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        peft_config=peft_config,
        args=training_args,
        tokenizer=tokenizer,
        max_seq_length=2048,
        dataset_text_field="text",  # Will be overridden by our custom dataset
        data_collator=lambda data: {
            'input_ids': torch.stack([d['input_ids'] for d in data]),
            'attention_mask': torch.stack([d['attention_mask'] for d in data]),
            'molecular_features': torch.stack([d['molecular_features'] for d in data]),
            'molecular_edges': torch.stack([d['molecular_edges'] for d in data])
        }
    )
    sft_trainer.train()
    
    # DPO Training
    dpo_trainer = DPOTrainer(
        model=model,
        args=TrainingArguments(
            output_dir="./dpo_agent",
            per_device_train_batch_size=1,
            gradient_accumulation_steps=16,
            learning_rate=5e-6,
            num_train_epochs=1,
            fp16=True
        ),
        train_dataset=train_dataset,  # Should be preference dataset in practice
        tokenizer=tokenizer,
        beta=0.1
    )
    dpo_trainer.train()
    
    return model

In [ ]:
agent.run(
    question=dataset[1]['question'],
    options=dataset[1]['options']
)